# Les classes
-----
<center style="font-size:20px;">
Loic Gouarin
</center>
&nbsp;
<center>
*Licence 3*
</center>
<center>
*Année 2017-2018*
</center>

-----

Jusqu'à présent nous avons vu les types de bases ainsi que les conteneurs de la STL ce qui nous permet de résoudre bons nombres de problèmes en C++. Néanmoins, il y a une chose importante qui manque. Nous avons parlé de l'importance du choix des noms de variables, des fonctions. Ceci dans un but de clarté pour l'utilisateur et les autres développeurs.

Les conteneurs sont des classes. Mais leur nom n'est pas très explicite lorsque l'on veut les utiliser pour un domaine bien spécifique. Prenons l'exemple de `std::vector` : ce conteneur peut représenter une position dans le plan, des données financières, des coordonnées GPS, ... 

Autour de ces représentations gravitent tout un tas d'opérateurs mathématiques permettant de les manipuler. Vous êtes en train d'utiliser le C++ pour faire des mathématiques et nous allons montrer dans la suite que nous pouvons créer nos propres objets permettant d'avoir une abstraction mathématique compréhensible immédiatement par les personnes qui lisent le code.

Prenons un exemple concret : nous souhaitons manipuler des polynômes. Un polynôme est constitué de symboles, de coefficients numériques. Nous pouvons factoriser, développer ou évaluer en certains points ces polynômes.

Supposons que nous avons cette première représentation

In [ ]:
std::vector<double> p1{1, 0, 1};
std::vector<double> p2{0, 2};

std::vector<double> p3 = mult(p1, p2);

std::cout << "Les coefficients de p3 sont: ";
for(auto c: p3)
    std::cout << c << " ";
std::cout << "\n";

Nous aurions la sortie suivante

```
Les coefficients de p3 sont: 0 2 0 2 
```
    
Supposons maintenant une deuxième représentation du problème

In [ ]:
symbol x;
poly p1 = 1 + x^2;
poly p2 = 2*x;

std::cout << "p1*p2 vaut " << p1*p2 << "\n";

Nous aurions la sortie suivante

```
p1*p2 vaut 2x^3 + 2x
```

Si nous donnons les deux codes à des utilisateurs potentiels, à votre avis lequel sera compréhensible de suite ? Il y a un autre avantage d'utiliser la deuxième représentation: vous donnez envie aux autres d'utiliser ce que vous avez fait.

Pour obtenir cette abstraction mathématique, que l'on peut également appeler [langage dédié](https://fr.wikipedia.org/wiki/Langage_d%C3%A9di%C3%A9), les classes en C++ jouent un rôle capital.

Dans la suite, nous simplifierons le problème en nous intéressant à la représentation des nombres rationnels et à leur manipulation. Pour rappel, l'ensemble des nombres rationnels est un corps commutatif, noté $\mathbb{Q}$. Sa définition est 

$$
\mathbb{Q} =\left\{\left.{\frac {m}{n}}\right|(m,n)\in \mathbb{Z } \times (\mathbb{Z } \setminus \{0\})\right\} 
$$

où $\mathbb{Z }$ est l'anneau des entiers relatifs.

## Premières notions d'une classe

Une classe en C++ peut être constituée d'attributs, de méthodes, de définitions de type et sous classes.

Commençons à implanter une classe pour les nombres rationnels.

In [1]:
class rational
{
public:
    int n, d;
    
    void simplify()
    {
        std::cout << "simplify\n";
    };
};

Ici les attributs de la classe `rational` sont

- `n` représentant le numérateur,
- `d` représentant le dénominateur.

Notre classe contient une méthode `simplify`.

Voici un exemple de son utilisation.

In [2]:
rational r{1, 2};

std::cout << r.n << "/" << r.d << "\n";

r.n = 3;

std::cout << r.n << "/" << r.d << "\n";

r.simplify();

1/2
3/2
simplify


`r` est une instance de la classe `rational`. Pour accéder à ses attributs et ses méthodes, il suffit d'utiliser l'opérateur `.`. Nous n'avons pas encore défini de constructeur. Le langage en fourni par défaut. Le fait d'avoir tout nos attributs en `public` permet également de les initilaiser en faisant

In [ ]:
rational r{1, 2};

L'initialisation se fait par odre d'apparition des attributs. Donc ici, 1 correspond à l'attribut `n` et 2 à l'attribut `d`.

Il est possible de choisir l'accessibilité des membres de la classe. Le langage en fournit trois

- `public` : accessible partout,
- `private`: accessible uniquement à l'intérieur de la classe,
- `protected` : accessible dans la classe elle-même et ses classes dérivées.

Dans notre premier exemple, nous avons tout mis en `public` ce qui fait que l'on peut accéder à `n`, `d` et `simplifiy` partout. Il peut être intéressant de mettre en `private` les éléments de base de la classe afin de ne pas les laisser accessible à l'utilisateur.

Réécrivons notre classe

In [1]:
class rational
{
public:
    
    void simplify()
    {
        std::cout << "simplify\n";
    };

private:
    
    int n, d;
    
};

Ici, nous avons toujours un constructeur par défaut.

In [2]:
rational r;

En revanche, il n'est plus possible de faire 

In [4]:
rational r{1, 2};

input_line_10:2:11: error: no matching constructor for initialization of 'rational'
 rational r{1, 2};
          ^~~~~~~
input_line_7:1:7: note: candidate constructor (the implicit copy constructor) not viable: requires 1
      argument, but 2 were provided
class rational
      ^
input_line_7:1:7: note: candidate constructor (the implicit move constructor) not viable: requires 1
      argument, but 2 were provided
input_line_7:1:7: note: candidate constructor (the implicit default constructor) not viable: requires 0
      arguments, but 2 were provided


Ni d'accéder aux attributs car ils sont maintenant privés.

In [5]:
r.n

input_line_11:2:4: error: 'n' is a private member of 'rational'
 r.n
   ^
input_line_7:11:9: note: declared private here
    int n, d;
        ^


Il est donc nécessaire de définir un constructeur et une interface pour accéder aux attributs privés. Le fait de créer le constructeur permet également de définir correctement le comportement que doit avoir notre classe par rapport aux paramètres d'entrées. Dans notre cas, un rationnel ne peut pas avoir un dénominateur nul.

Nous pouvons également créer des classes en utilisant le mot clé `struct`. Par exemple

In [ ]:
struct rational
{
    int n, d;
}

La différence entre `class` et `struct` est la suivante

- de base tous les éléments de `class` sont privés si on ne spécifie pas `public` ou `private`
- de base tous les éléments de `struct` sont publics si on ne spécifie pas `public` ou `private`

On préférera utiliser `class` lorsqu'au moins un des éléments est privé. Si tout est public, alors on choisira `struct`.

## Les constructeurs et affectations

Le langage fourni trois constructeurs

- le constructeur par défaut
- le constructeur par copie
- le constructeur par déplacement

Nous allons les voir plus en détail par la suite. Vous pouvez également définir vos propres constructeurs. Un constructeur n'a pas de type de retour et doit s'appeler par le nom de la classe. Nous allons ici créer notre propre constructeur.

In [1]:
class rational
{
public:  
        
    rational(const int n_, const int d_=1): n{n_}
    {
        std::cout << "constructeur\n";
        if (d_ == 0)
            throw std::invalid_argument::invalid_argument("d must be different to zero.");
        d = d_;
    }
        
    int get_n() const
    {
        return n;
    }
    
    void set_n(int n_)
    {
        n = n_;
    }

    int get_d() const
    {
        return d;
    }
    
    void set_d(int d_)
    {
        d = d_;
    }

    void simplify();

private:
    
    int n{0}, d{1};
    
};

In [3]:
rational r{1, 2};

In [4]:
r.get_d()

(int) 2


In [5]:
r.set_n(5);
r.get_n()

(int) 5


In [3]:
rational r1{2, 0};

Caught a std::exception!
d must be different to zero.


Plusieurs remarques :

- il est bon d'intialiser les attributs par des valeurs par défaut.
- dans le constructeur, nous pouvons directement affecter les attributs à des paramètres en mettant un deux points juste après la définition de celui-ci.
- nous n'avons pas donner le corps de la méthode `simplify`. Nous pouvons le faire dans le corps de la méthode (comme initialement montré) ou après.

Voyons comment faire pour donner le corps de `simplify` après la définition de la classe.

In [2]:
void rational::simplify()
{
    int pgcd = n;
    int tmp  = d;
    while (tmp != 0)
    {
        int r = pgcd%tmp;
        pgcd = tmp;
        tmp = r;
    }
    n /= pgcd;
    d /= pgcd;
}

In [3]:
rational r{6, 9};

constructeur


In [4]:
r.simplify();

In [5]:
r.get_n()

(int) 2


In [6]:
r.get_d()

(int) 3


La méthode `simplify` fait bien ce que l'on veut.

### Le constructeur par défaut

Le constructeur par défaut est un constructeur ne prenant pas de paramètres ou des valeurs par défaut pour chaque attribut de votre classe. Ici, le fait d'avoir spécifié un constructeur fait que nous n'avons plus de constructeur par défaut (si vous n'avez pas de constructeur, le langage en crée un pour vous comme lors de notre première implémentation de la classe `rational` lorsque tous nos attributs étaient `public`). Il est donc nécessaire de modifier un peu notre constructeur pour que l'on puisse faire à nouveau

In [4]:
rational r2;

Pour ce faire, il suffit juste de mettre l'ensemble des paramètres de notre constructeur avec des valeurs par défaut.

In [1]:
class rational
{
public:  
        
    rational(const int n_ = 0, const int d_ = 1): n{n_}
    {
        if (d_ == 0)
            throw std::invalid_argument::invalid_argument("d must be different to zero.");
        d = d_;
    }
        
    int get_n() const
    {
        return n;
    }
    
    void set_n(int n_)
    {
        n = n_;
    }

    int get_d() const
    {
        return d;
    }
    
    void set_d(int d_)
    {
        d = d_;
    }

    void simplify();

private:
    
    int n{0}, d{1};
    
};

In [4]:
rational r;

In [3]:
r.get_n()

(int) 0


In [4]:
r.get_d()

(int) 1


### Le constructeur par copie

Le constructeur par copie permet de faire la chose suivante

In [5]:
rational r_1(r), r_2{r};

Etant donné que nous n'avons pas précisé de constructeur par copie, le langage en construit un pour nous. Voici pourquoi la ligne du dessus s'exécute correctement. Voyons néanmoins son implantation pour comprendre ce que fait le langage par défaut. L'idée est donc de construire une copie d'une instance de `rational`. Le paramètre d'entrée en donc un `rational`. Etant donné que nous n'allons pas modifier ce paramètre, il est constant. De plus, nous le passons par référence. Ce qui nous donne

In [ ]:
class rational
{
public:  

    //...
    rational(const rational& r): n{r.n}, d{r.d}
    {
    }
    //...

};

Petite remarque: si votre constructeur de copie est aussi simple et peut-être fait par le langage alors ne le faite pas. On peut spécifier dans la classe que l'on utilise le constructeur par copie donné par le langage explicitement. Il suffit d'écrire

In [ ]:
class rational
{
public:  

    //...
    rational(const rational& r) = default;
    //...

};

Il est possible d'utiliser cette syntaxe pour l'ensemble des trois constructeurs ainsi que pour les opérateurs d'affectation.

#### Affectation par copie

Supposons que nous voulons dire qu'un rationnel est égal à un autre. Nous pourrions le faire de la manière suivante 

In [2]:
rational r1{1, 2}, r2;

In [6]:
r2.set_n(r1.get_n());
r2.set_d(r1.get_d());

In [7]:
r2.get_n()

(int) 1


In [8]:
r2.get_d()

(int) 2


Pas très joli tout ça. Nous préférerions faire 

In [9]:
r2 = r1;

Notez ici que nous effectuons une copie de tous les attributs de `r1` dans `r2`. Encore une fois la ligne du dessus est valide car le langage nous fournit un opérateur d'affectation si nous n'en avons pas.

Si nous devions le faire nous même, voici à quoi à ressemblerait

In [ ]:
class rational
{
public:  

    //...
    rational& operator=(const rational& r)
    {
        n = r.n;
        d = r.d;
        return *this;
    }
    //...

};

L'opérateur renvoie une référence avoir de pouvoir faire plusieurs affectations en même temps. `this` est un pointeur réprésentant l'objet lui même. Etant donné que nous voulons à la fin une référence, il est nécessaire de le déréférencer à l'aide de l'opérateur `*`.

Là encore, nous pouvons explicitement dire que nous utilisons l'opérateur d'affectation par défaut en écrivant

In [ ]:
class rational
{
public:  

    //...
    rational& operator=(const rational& r) = default;
    //...

};

### Le constructeur par déplacement

Ce constructeur permet à des `rvalue` de ne pas être copiées inutilement mais directement déplacées. Nous reviendrons plus en détail dans un prochain cours sur ce qu'est une `rvalue` mais dites vous que c'est un temporaire qui n'existe que ponctuellement. Lorsque ce temporaire est passé au constructeur, il est considéré à la sortie comme inutilisable.

Afin d'expliciter un peu plus ce qu'est un temporaire, voici un exemple

In [10]:
rational r3{rational{1, 2}};

Vous pouvez voir que `rational{1, 2}` n'est jamais affecté à une variable. Il n'existe donc que pour la construction de `r3` et après ne sert plus à rien. Par conséquent au lieu de le copier dans `r3` par le constructeur par copie vu précédemment, `r3` pourrait le voler.

In [ ]:
class rational
{
public:  

    //...
    rational(rational&& r): n{std::move(r.n)}, d{std::move(r.d)}
    {
    }
    //...

};

Encore une fois, on peut explicitement dire que l'on veut le constructeur par défaut.

In [ ]:
class rational
{
public:  

    //...
    rational(rational&& r) = default;
    //...

};

### Affectation par déplacement

Comme pour l'affectation par copie, il est possible de définir l'affectation par déplacement. 

In [ ]:
class rational
{
public:  

    //...
    rational& operator=(rational&& r)
    {
        n = std::move(r.n);
        d = std::move(d.n);
        return *this;
    }
    //...

};

Ou encore

In [ ]:
class rational
{
public:  

    //...
    rational& operator=(rational&& r) = default;
    //...

};

Reprenons l'ensemble de ces constructeurs et opérateurs d'affectation et regardons leur fonctionnement sur des cas concrets. 

In [1]:
class bidon
{
public: 
    
    bidon(int a1=0, int a2=0): a1{a1}, a2{a2}
    {
        std::cout << "constructeur par défaut\n";
    }
    
    bidon(const bidon& b): a1{b.a1}, a2{b.a2}
    {
        std::cout << "constructeur par copie\n";
    }
    
    bidon(bidon&& b): a1{std::move(b.a1)}, a2{std::move(b.a2)}
    {
        std::cout << "constructeur par déplacement\n";
    }
    
    bidon& operator=(const bidon& b)
    {
        std::cout << "affectation par copie\n";
        a1 = b.a1;
        a2 = b.a2;
        return *this;
    }

    bidon& operator=(bidon&& b)
    {
        std::cout << "affectation par déplacement\n";
        a1 = std::move(b.a1);
        a2 = std::move(b.a2);
        return *this;
    }

//private:

    int a1, a2;
}

In [2]:
bidon a{2, 3};

constructeur par défaut


In [3]:
bidon b(a);

constructeur par copie


In [4]:
&a.a1

(int *) 0x7f7b76bbd044


In [5]:
bidon c(std::move(a));

constructeur par déplacement


In [5]:
bidon d;

constructeur par défaut


In [6]:
d = bidon(1, 2);

constructeur par défaut
affectation par déplacement


In [7]:
d = c;

affectation par copie


Dans nos exemples, nous ne manipulons que des scalaires (ici des `int`). Le `move` n'a donc pas beaucoup d'intérêt. Il en a en revanche lorsque vous avez des conteneurs qui peuvent être de grande taille. Supposez que vous avez créé une `rvalue` d'un conteneur de type `std::vector` avec 1000 éléments et vous voulez utiliser l'opérateur d'affectation. Pourquoi faire une copie alors que c'est un temporaire et qu'il ne vous servira plus après. Il est donc préférable d'utiliser une affectation par déplacement permettant de pointer juste sur le bon espace mémoire et de rendre la `rvalue` dans un état non défini. C'est ce que fait l'implantation de l'affectation par déplacement de `std::vector`.

Dans certain cas, il est également utile de dire qu'une classe ne peut pas être copiée et/ou déplacée. A la même place où nous avons utilisé `default`, nous utiliserons `delete`. En voici un exemple

In [1]:
struct nocopy
{
    int i;
    nocopy() = default;
    nocopy(const nocopy&) = delete;
    nocopy& operator=(const nocopy&) = delete;
}

In [2]:
nocopy a;

Essayons de faire une copie.

In [3]:
nocopy b{a};

input_line_9:2:9: error: call to deleted constructor of 'nocopy'
 nocopy b{a};
        ^~~~
input_line_7:5:5: note: 'nocopy' has been explicitly marked deleted here
    nocopy(const nocopy&) = delete;
    ^


In [4]:
nocopy c;

Essayons de faire une copie par affectation.

In [5]:
c = a;

input_line_11:2:4: error: overload resolution selected deleted operator '='
 c = a;
 ~ ^ ~
input_line_7:6:13: note: candidate function has been explicitly deleted
    nocopy& operator=(const nocopy&) = delete;
            ^


In [1]:
struct nomovable
{
    int i;
    nomovable() = default;
    nomovable(nomovable&&) = delete;
    nomovable& operator=(nomovable&&) = delete;
}

Essayons de créer un `nomovable` à partir d'une `rvalue`.

In [2]:
nomovable am = nomovable();

input_line_8:2:12: error: call to deleted constructor of 'nomovable'
 nomovable am = nomovable();
           ^    ~~~~~~~~~~~
input_line_7:5:5: note: 'nomovable' has been explicitly marked deleted here
    nomovable(nomovable&&) = delete;
    ^


In [3]:
nomovable am, bm;

Essayons de faire une affectation par déplacement.

In [4]:
am = std::move(bm);

input_line_10:2:5: error: overload resolution selected deleted operator '='
 am = std::move(bm);
 ~~ ^ ~~~~~~~~~~~~~
input_line_7:6:16: note: candidate function has been explicitly deleted
    nomovable& operator=(nomovable&&) = delete;
               ^
input_line_7:1:8: note: candidate function (the implicit copy assignment operator) has been implicitly deleted
struct nomovable
       ^


## `static` et `inline`

L'utilisation `static` et de `inline` peut être très intéressant. Ils n'ont néanmoins rien à voir entre eux.

- `static` signifie que l'attribut n'est pas associé à une instance particulière mais à la classe elle-même. Par conséquent chaque instance partage cet attribut.

- `inline` permet de remplacer directement l'appel de la méthode ou de la fonction par son code. Ceci peut permettre un certain nombre d'optimisation. En effet, lorsque l'on appelle une fonction le compilateur ne sait pas ce qu'elle fait. En revanche si on utilise `inline`, la fonction n'est plus appelée et son code est directement injecté dans le corps du programme. Le compilateur a alors tous les éléments pour comprendre la suite logique de notre algorithme.

Dans notre exemple, nous allons utiliser ces deux fonctionalités.

- `static` pour compter le nombre de rationnels que nous avons construit.

- `inline` pour la méthode `simplify`

A cause d'un bug dans `cling`, nous sommes obligés d'écrire un script C++ pour l'utlisation d'un `static` comme attribut.

In [13]:
%%file rational.cpp

#include <iostream>
#include <stdexcept>

class rational
{
public:  
    
    static std::size_t nb_rational;
    
    rational(const int n_ = 0, const int d_ = 1): n{n_}
    {
        nb_rational++;
        if (d_ == 0)
            throw std::invalid_argument("d must be different to zero.");
        d = d_;
        simplify();
    }
        
    rational& operator=(const rational& r)
    {
        rational::nb_rational++;
        n = r.n;
        d = r.d;
        return *this;
    }

    rational(const rational& r): n{r.n}, d{r.d}
    {
        rational::nb_rational++;
    }
    
    ~rational()
    {
        rational::nb_rational--;
    }
    
    int numerator() const
    {
        return n;
    }
    
    void numerator(int n_)
    {
        n = n_;
    }

    int denominator() const
    {
        return d;
    }
    
    void denominator(int d_)
    {
        d = d_;
    }

    inline void simplify()
    {
        int pgcd = n;
        int tmp  = d;
        while (tmp != 0)
        {
            int r = pgcd%tmp;
            pgcd = tmp;
            tmp = r;
        }
        n /= pgcd;
        d /= pgcd;        
    }

private:
    
    int n{0}, d{1};
    
};

std::size_t rational::nb_rational = 0; // Il est nécessaire de l'initialiser

void f()
{
    rational r;
    std::cout << "Dans f: " << rational::nb_rational << "\n";
}

int main()
{
    rational r1, r2;
    std::cout << "Avant f: " << rational::nb_rational << "\n";
    f();
    std::cout << "Apres f: " << rational::nb_rational << "\n";
    return 0;
}

Overwriting rational.cpp


In [14]:
! g++ rational.cpp

In [15]:
! ./a.out

Avant f: 2
Dans f: 3
Apres f: 2


## La surchage d'opérateurs

Le langage C++ permet de surcharger un bon nombre d'opérateurs. Nous allons voir ici comment surcharger l'addition, la soustraction, la multiplication, la division et la puissance pour notre classe `rational`.

Il est assez facile de surcharger un opérateur et vous l'avez déjà vu pour l'opérateur `=`. Voici ce que ça donne pour les autres opérateurs.

In [ ]:
class rational
{
public:  
    
    /// ...
    rational& operator+=(const rational& r)
    {
        n = n*r.d + d*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator-=(const rational& r)
    {
        n = n*r.d - d*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator*=(const rational& r)
    {
        n = n*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator/=(const rational& r)
    {
        n = n*r.d;
        d = d*r.n;
        simplify();
        return *this;
    }

    rational& operator^=(const double& r)
    {
        n = std::pow(n, r);
        d = std::pow(d, r);
        simplify();
        return *this;
    }
/// ...
};

Cette implémentation ne définit que les opérateurs `+=`, `-=`, `*=`, `/=` et `^=`. Mais ce que nous voudrions pouvoir faire est la chose suivante

In [ ]:
rational r1{1, 3}, r2{1, 2};
rational r3 = r1 + r2;

Il est donc necessaire de définir d'autres opérateurs.

In [ ]:
rational& operator+(const rational& r1, const rational& r2)
{
    rational output(r1);
    output += r2;
    return output;
}

rational& operator-(const rational& r1, const rational& r2)
{
    rational output(r1);
    output -= r2;
    return output;
}

rational& operator*(const rational& r1, const rational& r2)
{
    rational output(r1);
    output *= r2;
    return output;
}

rational& operator/(const rational& r1, const rational& r2)
{
    rational output(r1);
    output /= r2;
    return output;
}

rational& operator^(const rational& r1, const double& d)
{
    rational output(r1);
    output ^= d;
    return output;
}

ostream& operator<<(ostream& out, const rational& r)
{
    out << r.numerator() << "\\" << r.denominator();
    return out;
}

La encore, pour tester toutes ces méthodes nous devons les écrires dans un script.

In [55]:
%%file rational.cpp

#include <iostream>
#include <stdexcept>
#include <cmath>

class rational
{
public:  
    
    rational(const int n_ = 0, const int d_ = 1): n{n_}
    {
        if (d_ == 0)
            throw std::invalid_argument("d must be different to zero.");
        d = d_;
        simplify();
    }        
    
    rational& operator+=(const rational& r)
    {
        n = n*r.d + d*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator-=(const rational& r)
    {
        n = n*r.d - d*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator*=(const rational& r)
    {
        n = n*r.n;
        d = d*r.d;
        simplify();
        return *this;
    }

    rational& operator/=(const rational& r)
    {
        n = n*r.d;
        d = d*r.n;
        simplify();
        return *this;
    }

    rational& operator^=(const double& r)
    {
        n = std::pow(n, r);
        d = std::pow(d, r);
        simplify();
        return *this;
    }

    int numerator() const
    {
        return n;
    }
    
    void numerator(int n_)
    {
        n = n_;
    }

    int denominator() const
    {
        return d;
    }
    
    void denominator(int d_)
    {
        d = d_;
    }

    inline void simplify()
    {
        int pgcd = n;
        int tmp  = d;
        while (tmp != 0)
        {
            int r = pgcd%tmp;
            pgcd = tmp;
            tmp = r;
        }
        n /= pgcd;
        d /= pgcd;        
    }

private:
    
    int n{0}, d{1};
};

rational& operator+(const rational& r1, const rational& r2)
{
    rational output{r1};
    return output += r2;
}

rational& operator-(const rational& r1, const rational& r2)
{
    rational output{r1};
    return output -= r2;
}

rational& operator*(const rational& r1, const rational& r2)
{
    rational output{r1};
    return output *= r2;
}

rational& operator/(const rational& r1, const rational& r2)
{
    rational output{r1};
    return output /= r2;
}

rational& operator^(const rational& r1, const double& d)
{
    rational output{r1};
    return output ^= d;
}

std::ostream& operator<<(std::ostream& out, const rational& r)
{
    out << r.numerator() << "/" << r.denominator();
    return out;
}

int main()
{
    rational r1{2, 3}, r2{7, 6};
    double d = 3;
    //rational r3 = r1^3;
    std::cout << "r1 : " << r1 << "\n";
    std::cout << "r2 : " << r2 << "\n";
    std::cout << r1 << " + " << r2 << " = " << r1 + r2 << "\n";
    std::cout << r1 << " - " << r2 << " = " << r1 - r2 << "\n";
    std::cout << r1 << " * " << r2 << " = " << r1 * r2 << "\n";
    std::cout << r1 << " / " << r2 << " = " << r1 / r2 << "\n";
    std::cout << r1 << " ^ " <<  d << " = " << (r1^d)      << "\n";
    return 0;
}

Overwriting rational.cpp


In [56]:
! g++ rational.cpp

In [57]:
! ./a.out

r1 : 2/3
r2 : 7/6
2/3 + 7/6 = 11/6
2/3 - 7/6 = 1/-2
2/3 * 7/6 = 7/9
2/3 / 7/6 = 4/7
2/3 ^ 3 = -786671984/32766
